In [1]:
import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [2]:
from ray.rllib.algorithms import ppo
from envs import TicTacToeEnv

2023-08-02 11:33:23,005	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [3]:
def train(max_episodes=10):
    config = ppo.PPOConfig().environment(env=TicTacToeEnv, env_config={})
    config = config.rl_module(_enable_rl_module_api=False).training(_enable_learner_api=False)
    algo = config.build()
    for i in range(max_episodes):
        result = algo.train()
        print("episode reward mean: ", result["episode_reward_mean"])

    return algo


def evaluate(algo):
    env = TicTacToeEnv()
    observation, info = env.reset()
    terminated = False
    while not terminated:
        # action = env.action_space.sample()
        action = algo.compute_single_action(observation)
        observation, reward, terminated, truncated, info = env.step(action)
        print("observation: ", observation)

        if terminated or truncated:
            observation, info = env.reset()

    env.close()

In [4]:
# algo = train(max_episodes=50)
# algo.export_policy_model("policy_model", onnx=18)

In [5]:
import onnx
onnx_model = onnx.load("policy_model/model.onnx")
onnx.checker.check_model(onnx_model)

In [6]:
onnx.checker.check_model(onnx_model, full_check=True)

In [12]:
import onnxruntime as ort
import numpy as np

env = TicTacToeEnv()

# obs = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])
obs = np.zeros((1, 9)).astype(np.int32)
state_ins = np.zeros((1,)).astype(np.float32)
# 
obs, info = env.reset()
obs = np.array(obs).astype(np.int32).reshape(1, -1)
print(obs.shape)
print(obs)
ort_sess = ort.InferenceSession('policy_model/model.onnx')
outputs = ort_sess.run(None, {'obs': obs, 'state_ins': state_ins})
outputs

(1, 9)
[[0 0 0 0 0 0 0 0 0]]


[array([[ 0.7512674, -3.3365052, -1.4412379, -1.9218855,  1.9381831,
         -1.0040662, -0.7864778, -2.21011  ,  9.571112 ]], dtype=float32),
 array([0.], dtype=float32)]

In [ ]:
outputs[0]

array([[ 0.7512674, -3.3365052, -1.4412379, -1.9218855,  1.9381831,
        -1.0040662, -0.7864778, -2.21011  ,  9.571112 ]], dtype=float32)

In [ ]:
policy = algo.get_policy()
print(policy)

PPOTorchPolicy


In [ ]:
policy.observation_space_struct

Box(0, 2, (9,), int32)

In [ ]:
env.action_space.sample()

1